## Look at one book closely

Here, I examine quotations from the Song of Solomon (although I could easily do any book).

In [3]:
import time, pickle, glob, os
from matching_functions import *

BIBLE_SHINGLE_FOLDER = '/home/spenteco/text_reuse.HOME/bible_pickles/'
EEBO_SHINGLE_FOLDER = '/home/spenteco/0/eebo_shingled/'
RESULTS_FOLDER = '/home/spenteco/0/bible_matches/'
METADATA_FILE = 'metadata/EEBO_metadata.tsv'

metadata = load_metadata(METADATA_FILE)
            
def get_metadata_for_id(tcp_id):
    
    year = ''
    author = ''
    title = ''

    try:
        year = metadata[tcp_id]['year']
        author = metadata[tcp_id]['author'].split('|')[0]
        title = metadata[tcp_id]['title']
    except KeyError:
        pass
    
    return year, author, title

### get_specific_bible_quotations is a . . . 

 . . . function to dig all quotations from a book of the bible (and perhaps, from a chapter and verse) from the mass of results.  Here, I'm picking out Song of Solomon in its entirety.

In [4]:
def get_specific_bible_quotations(book, chapter=None, verse=None):

    quotation_data = []

    for p in glob.glob('/home/spenteco/0/bible_matches/*.pickle'):

        tcp_id = p.split('/')[-1].split('.')[0]

        f = open(p, 'rb')
        data = pickle.load(f)
        f.close()

        book_matches = []

        for match in data:

            reference = []
            for pn, p in enumerate(match[1].split('.')):
                if pn == 0:
                    reference.append(p.replace('_', ' '))
                else:
                    reference.append(int(p))
            reference = tuple(reference)

            if reference[0] != book:
                continue
            if chapter != None and chapter != reference[1]:
                continue
            if verse != None and verse != reference[2]:
                continue
            
            book_matches.append(match)

        if len(book_matches) > 0:
            quotation_data.append([tcp_id, book_matches])
            
    return quotation_data

# --------------------------------------------------------------------

quotation_data = get_specific_bible_quotations('Song of Solomon')
    
print('len(quotation_data)', len(quotation_data))

len(quotation_data) 1512


### Make an aligned Bible

This cell produces an aligned Bible for our three versions.  It can produce it for a whole book, or for a chapter.

Here, I'm producing it for the whole of Song of Solomon.  The results are interesting . . . the Geneva verse divisions not always map to the KJV verse divisions.  For each version, I print the original text plus the non-space, non-stopword lemma, since matching starts with them.

In [11]:
from IPython.display import HTML, display
import tabulate

def make_aligned_bible(book, chapter=None):

    aligned_verses = {}

    for tcp_id in ['A10675', 'A97378', 'KJV_OTA']:

        f = open(BIBLE_SHINGLE_FOLDER + tcp_id + '.pickle', 'rb')
        verses = pickle.load(f)
        f.close()

        for verse in verses:
            
            reference = []
            for pn, p in enumerate(verse['reference'].split('.')):
                if pn == 0:
                    reference.append(p.replace('_', ' '))
                else:
                    reference.append(int(p))
            reference = tuple(reference)

            if reference[0] != book:
                continue
            if chapter != None and chapter != reference[1]:
                continue
                
            text = ''.join(verse['tokens'])
            lemma = ' '.join(verse['non_space_lemmas'])
            
            if reference not in aligned_verses:
                aligned_verses[reference] = {'A10675': ['', ''], 'A97378': ['', ''], 'KJV_OTA': ['', ''], }

            aligned_verses[reference][tcp_id][0] = text
            aligned_verses[reference][tcp_id][1] = lemma

    all_output_lines = [['book', 'chapter', 'verse', 'A10675 text', 'lemma', 
                                                     'A97378 text', 'lemma', 
                                                     'KJV_OTA text', 'lemma']]

    for reference in sorted(aligned_verses.keys()):
        output_line = [reference[0], reference[1], reference[2]]
        for tcp_id in ['A10675', 'A97378', 'KJV_OTA']:
            try:
                output_line.append(aligned_verses[reference][tcp_id][0])
                output_line.append(aligned_verses[reference][tcp_id][1])
            except KeyError:
                output_line.append('')
                output_line.append('')
        all_output_lines.append(output_line)

    display(HTML(tabulate.tabulate(all_output_lines, tablefmt='html')))
    
# --------------------------------------------------------------------------

make_aligned_bible('Song of Solomon')
#make_aligned_bible('Song of Solomon', chapter=2)

book,chapter,verse,A10675 text,lemma,A97378 text,lemma,KJV_OTA text,lemma
Song of Solomon,1,1,1 LEt hī kisse me with the kisses of his mouthe: for thy love is better then wine.,let hī kiss kiss mouth love better wine,"THe song of songs, which is Solomons.",song song solomons,"The song of songs, which is Solomon's.",song song solomon
Song of Solomon,1,2,2 Because of the savour of thy good ointments thy name (is as) an oyntment powred out: therfore the virgines love thee.,savour good ointment name ointment pour therefore virgin love,2 Let him kiss me with the kisses of his mouth: for thy love is better then wine.,let kiss kiss mouth love better wine,Let him kiss me with the kisses of his mouth: for thy love is better than wine.,let kiss kiss mouth love better wine
Song of Solomon,1,3,3 Drawe me: we will runne after thee: the King hath broght me into his chambers: we wil reioyce and be glad in thee: we will remember thy love more then wyne: the righteous do love thee.,draw run king bring chamber rejoice glad remember love wine righteous love,"3 Because of the savour of thy good ointments, thy name is as ointment poured forth, therefore do the virgins love thee.",savour good ointment name ointment pour forth therefore virgin love,"Because of the savour of thy good ointments thy name is as ointment poured forth, therefore do the virgins love thee.",savour good ointment name ointment pour forth therefore virgin love
Song of Solomon,1,4,"4 I am blacke, ô daughters of jerusalē, but comelie, as the frutes of Kedár, (&) as the curtines of Salomón.",black daughter jerusalē comely fruit kedár curtine salomón,"4 Draw me, we will run after thee: the king hath brought me into his chambers: we will be glad and rejoyce in thee, we will remember thy love more then wine: the upright love thee.",draw run king bring chamber glad rejoice remember love wine upright love,"Draw me, we will run after thee: the king hath brought me into his chambers: we will be glad and rejoice in thee, we will remember thy love more than wine: the upright love thee.",draw run king bring chamber glad rejoice remember love wine upright love
Song of Solomon,1,5,5 Regarde ye me not because I am blacke: for the sunne hathe loked upon me. The sonnes of my mother were angrie against me they made me the keper of the vines: but I kept not mine owne wine.,regard not black sun look upon son mother angry make keeper vine keep not wine,"5 I am black, but comely, O ye daughters of Jerusalem, as the tents of Kedar, as the curtains of Solomon.",black comely oh daughter jerusalem tent kedar curtain solomon,"I am black, but comely, O ye daughters of Jerusalem, as the tents of Kedar, as the curtains of Solomon.",black comely oh daughter jerusalem tent kedar curtain solomon
Song of Solomon,1,6,"6 Shewe me, (ô thou,) whome my soule loveth, where thou fedest, where thou liest at noone: for why shuld I be as she that turneth aside to the flockes of thy cōpaniōs?",show soul love fede lie noon shall turn aside flock cōpaniō,"6 Look not upon me, because I am black, because the sun hath looked upon me: my mothers children were angry with me, they made me the keeper of the vineyards, but mine own vineyard have I not kept.",look not upon black sun look upon mother child angry make keeper vineyard vineyard not keep,"Look not upon me, because I am black, because the sun hath looked upon me: my mother's children were angry with me; they made me the keeper of the vineyards; but mine own vineyard have I not kept.",look not upon black sun look upon mother child angry make keeper vineyard vineyard not keep
Song of Solomon,1,7,"7 If thou knowe not, ô thou the falrest among, women get thee forthe by the steppes of the flocke, and fede thy kiddes by the tents of the shepherdes.",know not falrest among woman get forth step flock feed kid tent shepherdess,"7 Tell me, O thou whom my soul loveth, where thou seedest, where thou makest thy flock to rest at noon: for why should I be as one that turneth aside by th

### Basic counts

Which texts quote which versions of the Song of Solomon?  How often?

In [41]:
from IPython.display import HTML, display
import tabulate

all_output_lines = [['tcp_id', 'year', 'title', 'A10675', 'A97378', 'KJV_OTA']]

for q in quotation_data:
    
    source_counts = {'A10675': 0, 'A97378': 0, 'KJV_OTA': 0}
    
    for match in q[1]:
            
        source_file = match[0].split('/')[-1].split('.')[0]

        verse = []
        for pn, p in enumerate(match[1].split('.')):
            if pn == 0:
                verse.append(p.replace('_', ' '))
            else:
                verse.append(int(p))
        verse = tuple(verse)
        
        source_counts[source_file] += 1
        
    all_output_lines.append([q[0], 
                             get_metadata_for_id(q[0])[0],
                             get_metadata_for_id(q[0])[2][:80],
                             source_counts['A10675'], 
                             source_counts['A97378'], 
                             source_counts['KJV_OTA']])
        
display(HTML(tabulate.tabulate(all_output_lines, tablefmt='html')))

tcp_id,year,title,A10675,A97378,KJV_OTA
A10675,1562,The Bible and Holy Scriptures conteyned in the Olde and Newe Testament. Translat,121,21,21
A45313,1647,"Satans fiery darts quenched, or, Temptations repelled in three decades : for the",0,2,2
A62715,1674,"A call to the Shulamite, or to the scattered and divided members of the church d",1,2,2
A92967,1664,"Christ under the law with the times of the Gospel, and fullness thereof.",2,1,1
A52417,1687,"A collection of miscellanies consisting of poems, essays, discourses, and letter",0,1,2
A63826,1656,"A good day vvell improved, or Five sermons upon Acts 9. 31 Two of which were pre",0,2,2
A40725,1673,"Concio ad magistratum a nations honour, and a nations dishonour, or, A kingdoms",1,1,1
A38823,1657,"The Gospel treasury opened, or, The holiest of all unvailing discovering yet mor",0,6,6
A51306,1656,The life and doctrine of ovr Savior Iesvs Christ. The first part with short refl,1,1,1
A87543,1696,"The liberty of prayer asserted, and garded [sic] from licentiousness by a minist",0,1,1


### A function to count . . . 

 . . . verse by verse, the number of times a section of the bible is quoted.

In [12]:
from IPython.display import HTML, display
import tabulate

def count_verses_for_source(source_tcp_id, chapter=None):

    verse_counts = {}

    for q in quotation_data:

        for match in q[1]:

            source_file = match[0].split('/')[-1].split('.')[0]
            
            if source_file != source_tcp_id:
                continue

            verse = []
            for pn, p in enumerate(match[1].split('.')):
                if pn == 0:
                    verse.append(p.replace('_', ' '))
                else:
                    verse.append(int(p))
            verse = tuple(verse)
            
            if chapter != None and chapter != verse[1]:
                continue

            if verse not in verse_counts:
                verse_counts[verse] = {'n': 0, 'text': match[2]}

            verse_counts[verse]['n'] += 1

    all_output_lines = all_output_lines = [['tcp_id', 'book', 'chapter', 'verse', 'n\nquoted', 'text']]

    for reference in sorted(verse_counts.keys()):
        output_line = [source_tcp_id, reference[0], reference[1], reference[2], 
                       verse_counts[reference]['n'], verse_counts[reference]['text']]
        all_output_lines.append(output_line)

    display(HTML(tabulate.tabulate(all_output_lines, tablefmt='html')))
    
# ---------------------------------------------------------------------------

### A set of three listings . . . 

 . . . one for each version of the bible, which shows how often each verse in chapter 2 of Song of Solomon is quoted.

In [13]:
count_verses_for_source('A10675', chapter=2)

tcp_id,book,chapter,verse,n quoted,text
A10675,Song of Solomon,2,1,29,"1 I Am the rose of the field, (and) the lilie of the valleis."
A10675,Song of Solomon,2,2,101,"2 Like a lilie among the thornes, so (is) my love among the daughters."
A10675,Song of Solomon,2,3,1,"3 Like the apple tre amōg the trees of the forest: so (is) my welbeloved amōg the sonnes (of mē:) under his shado we had I delite, and sate downe: and hys frute (was) swete unto my mouth."
A10675,Song of Solomon,2,4,1,"4 He broght me into the wine celler, & love (was) his ban ner over me."
A10675,Song of Solomon,2,5,1,"5 Stay me with flagons, and cōfort me with apples: for I am sicke of love."
A10675,Song of Solomon,2,6,221,"6 His left hand (is) under mine head, & hys right hand doeth imbrace me."
A10675,Song of Solomon,2,7,1,"7 I charge you, ô daughters of jerusalēm, by the roes and by the hindes of the field, that ye stirre not up, nor waken (my) love until she please."
A10675,Song of Solomon,2,8,31,"8 (It is) the voice of my welbeloved behold, he cometh leaping by the mountaines, & skipping by the hilles."
A10675,Song of Solomon,2,9,23,"9 My welbeloved is like a roe, or a yong heart: lo, he standeth behinde our wall, loking forthe of the windowes, shewing him selfe him selfe through the grates."
A10675,Song of Solomon,2,10,53,"10 My welbeloved spake & said unto me, Arise, my love, my faire one, & come thy way"


In [14]:
count_verses_for_source('A97378', chapter=2)

tcp_id,book,chapter,verse,n quoted,text
A97378,Song of Solomon,2,1,127,"I Am the rose of Sharon, and the lily of the valleys."
A97378,Song of Solomon,2,2,110,"2 As the lily among thorns, so is my love among the daughters."
A97378,Song of Solomon,2,3,61,"3 As the apple-tree among the trees of the wood, so is my beloved among the sons, I sat down under his shadow with great delight, and his fruit was sweet to my taste."
A97378,Song of Solomon,2,4,32,"4 He brought me to the banquetting-house, and his banner over me was love."
A97378,Song of Solomon,2,5,165,"5 Stay me with flagons, comfort me with apples; for I am sick of love."
A97378,Song of Solomon,2,6,221,"6 His left hand is under my head, and his right hand doth embrace me."
A97378,Song of Solomon,2,7,55,"7 I charge you, O ye daughters of Jerusalem, by the roes, and by the hinds of the field, that ye stir not up, not awake my love, till he please."
A97378,Song of Solomon,2,8,48,"8 ¶ The voice of my beloved! behold, he cometh leaping upon the mountains, skipping upon the hills."
A97378,Song of Solomon,2,9,26,"9 My beloved is like a roe, or a young hart: behold, he standeth behinde our wall, he looketh forth at the windows, shewing himself through the lattess."
A97378,Song of Solomon,2,10,58,"10 My beloved spake, and said unto me, Rise up my love, my fair one, and come away."


In [15]:
count_verses_for_source('KJV_OTA', chapter=2)

tcp_id,book,chapter,verse,n quoted,text
KJV_OTA,Song of Solomon,2,1,127,"I am the rose of Sharon, and the lily of the valleys."
KJV_OTA,Song of Solomon,2,2,110,"As the lily among thorns, so is my love among the daughters."
KJV_OTA,Song of Solomon,2,3,62,"As the apple tree among the trees of the wood, so is my beloved among the sons. I sat down under his shadow with great delight, and his fruit was sweet to my taste."
KJV_OTA,Song of Solomon,2,4,72,"He brought me to the banqueting house, and his banner over me was love."
KJV_OTA,Song of Solomon,2,5,165,"Stay me with flagons, comfort me with apples: for I am sick of love."
KJV_OTA,Song of Solomon,2,6,221,"His left hand is under my head, and his right hand doth embrace me."
KJV_OTA,Song of Solomon,2,7,90,"I charge you, O ye daughters of Jerusalem, by the roes, and by the hinds of the field, that ye stir not up, nor awake my love, till he please."
KJV_OTA,Song of Solomon,2,8,48,"The voice of my beloved! behold, he cometh leaping upon the mountains, skipping upon the hills."
KJV_OTA,Song of Solomon,2,9,26,"My beloved is like a roe or a young hart: behold, he standeth behind our wall, he looketh forth at the windows, shewing himself through the lattice."
KJV_OTA,Song of Solomon,2,10,58,"My beloved spake, and said unto me, Rise up, my love, my fair one, and come away."


### Extract just quotes of Chapter 2, Verse 15

**Geneva**: *Take us the foxes, the litle foxes, whiche destroy the vines: for our vines (have) smale grapes.*

**KJV**: *Take us the foxes, the little foxes, that spoil the vines: for our vines have tender grapes.*

The verse is interest me because the two versions differ slightly, because the match counts are so different (17 with Geneva, 64 with KJV), and because, in context, they have no obvious meaning--they're all metaphor.

In [69]:
sos_2_15 = get_specific_bible_quotations('Song of Solomon', chapter=2, verse=15)

print(len(sos_2_15))
print(sos_2_15[0])

72
['A10675', [['/home/spenteco/text_reuse.HOME/bible_pickles/A10675.pickle', 'Song of Solomon.2.15', '15 Take us the foxes, the litle foxes, whiche destroy the vines: for our vines (have) smale grapes.', ['Take us the foxes, the litle foxes, whiche destroy the vines: for our vines (have) smale grapes', 'Take us the foxes, the litle foxes, whiche destroy the vines: for our vines (have) smale grapes', [2, 39], [1059736, 1059773]]]]]


### Get context for a quotation

Here, I demonstrate getting the context--the surrounding text--for a quotation, using quotations from SoS 2:15.

If I had to generalize, I would say that when quoting the verse, writers interpret the vines as being the church, and the foxes as being heretics, etc.  Or vines as being individual believers and foxes as temptation, error, and so forth.

But there's unusual context, too.  In *The London printers lamentation*, the vines are the printing industry.  In the *Memorials of the English affairs*, the vines seem to be Great Britain, with Commons and Lords being the lover found in SoS.  Odd.

In [95]:
from IPython.display import HTML, display
import tabulate

all_output_lines = [['tcp_id', 'year', 'author', 'title', 'source', 'context']]

for matching_text in sos_2_15:
    
    for match in matching_text[1]:
    
        source = match[0].split('/')[-1].split('.')[0]
        
        if source == 'KJV_OTA':
            continue
            
        if source == 'A10675':
            source = 'Geneva'
        else:
            source = 'KJV'

        f = open(EEBO_SHINGLE_FOLDER + matching_text[0] + '.pickle', 'rb')
        matching_text_data = pickle.load(f)
        f.close()
        
        matching_text_location = match[3][3]
        
        from_location = matching_text_location[0] - 100
        to_location = matching_text_location[1] + 100
        
        if from_location < 0:
            from_location = 0
        if to_location > len(matching_text_data['tokens']):
            to_location = len(matching_text_data['tokens'])
        
        context = ''.join(matching_text_data['tokens'][from_location: to_location])

        all_output_lines.append([matching_text[0], 
                              get_metadata_for_id(matching_text[0])[0], 
                              get_metadata_for_id(matching_text[0])[1][:30], 
                              get_metadata_for_id(matching_text[0])[2][:50],
                              source,
                              context])
        
display(HTML(tabulate.tabulate(all_output_lines, tablefmt='html')))

tcp_id,year,author,title,source,context
A10675,1562,"Whittingham, William, d. 1579.",The Bible and Holy Scriptures conteyned in the Old,Geneva,"arise my love, my faire one, and come away.14 My doove, that art in the holes of the rocke, in the secret places of the staires, shewe me thy sight, let me heare thy voice: for thy voyce is swete: and thy sight comelie.15 Take us the foxes, the litle foxes, whiche destroy the vines: for our vines (have) smale grapes.16 My welbeloved (is) mine, and I am his: he fedeth among the lilies,17 Vntil the daye breake, and the shadowes flee away: returne, my welbeloved, (and) be like a roe, or a yong hart upon the mountaines of Béther.CHAP. III.1"
A64958,1695,"Vincent, Nathanael, 1639?-1697",The cure of distractions in attending upon God in,KJV,"; and truly the propagators, of lesser Errors do a great deal of harm, they are called little Foxes, and are often very prejudicial especially to young Converts, and prove a great hindrance to the good work begun in them: Therefore you read, Cant. 2. 15. Take us the Foxes, the little Foxes that spoil the Vines, for our Vines have tender Grapes. Errours, how do they excite the zeal of the Soul! And though they are but about smaller matters, yet they so command the Tongue and Thoughts, that they are more talked of and minded, than the great things of Law and Gospel. Erroneous Opinions do so"
A27999,1700,"Patrick, Simon, 1626-1707.",A paraphrase upon the books of Ecclesiastes and th,KJV,"praises and thanksgivings, and I will save and deliver thee; for I love thee inseparably, who art most amiable in my eyes, whose prayers and praises I delight to hear, and to behold my own Image that is formed in thee. See Annot. [n]. 15. Take us the foxes, the little foxes that spoil the vines: for our vines have tender grapes.15. And therefore I require all those that act by authority from me, and to whom I have committed the care of my Church, to use their early diligence also to discover and confute the sophistry of Deceivers, who craftily insinuate their false Doctrines into weak and"
B01215,1626,"Dyke, Jeremiah, 1584-1639.","Good conscience: or a treatise shewing the nature,",KJV,"send forth a stinking savour. Eccl. 10. 1. & so doth a little folly, though but little doe a great deale of hurt. And therefore a good cōscience lives by Salomons rule, Give not water a passage, no not a little. And takes not onely the Foxes, but the little Foxes, which spoyle not only the Vines, but the tender Grapes. Cant. 2. 15. It knowes a little will make way for much. Pharaoh is content that the people, the men should go Sacrifice, Exod. 10. but their little ones should not goe. he knew if hee had but their little ones with him, he should"
A55363,1683,"Poole, Matthew, 1624-1679.",Annotations upon the Holy Bible. Vol. I wherein th,KJV,"my fair one, and come away.14 O my dove, that art in the clefts of the rock, in the secret places of the stairs, let me see thy countenance, let me hear thy voice; for sweet is thy voice, and thy countenance is comely.15 Take us the foxes, the little foxes, that spoil the vines: for our vines have tender grapes.16 My beloved is mine, and I am his: he feedeth among the lillies.17 Until the day break, and the shadows flee away: turn my beloved, and be thou like a roe, or a young hart upon the mountains of Bether.CHAP. III.1 BY"
A09339,1600,"Perkins, William, 1558-1602.",A golden chaine: or The description of theologie c,Geneva,", Put forth thy riddle that we may heare it. 14. And he saide unto them, Out of the eater came meate, and out of the strong came sweetnesse: and they could not in t•ree daies expound the riddle. And hunting of wild beasts. Cant. 2.15. Take us the foxes, the little foxes which destroy the vines: for our vines have small grapes. Lastly, the searching out or the contemplation of the works of God. 1. King. 4.33. And he spake of trees from the cedar tree that is in Lebanon, euē unto the hyssop that springeth out of the wal: he spak• also 